# Art Finder

<div class="usecase-authors"><b>Authored by: </b>Harley Ngo</div>

<div class="usecase-date"><b>Date: </b> March 2023</div>

<div class="usecase-duration"><b>Duration:</b> 90 mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b> Level: </b>Intermediate</div>
    <div class="usecase-skill"><b> Pre-requisite Skills: </b>Python

<div class="usecase-subsection-blurb">
  <i>Dataset 1:</i> Outdoor artworks
  <br>
  <a href="https://data.melbourne.vic.gov.au/explore/dataset/outdoor-artworks/table/" target="_blank">Dataset Link</a>
  <br>
</div>
<br>

<div class="usecase-subsection-blurb">
  <i>Dataset 2:</i> Public artworks, fountains and monuments
  <br>
  <a href="https://data.melbourne.vic.gov.au/explore/dataset/public-artworks-fountains-and-monuments/table/" target="_blank">Dataset Link</a>
  <br>
</div>
<br>

<div class="usecase-subsection-blurb">
  <i>Dataset 3:</i> Public memorials and sculptures
  <br>
  <a href="https://data.melbourne.vic.gov.au/explore/dataset/public-memorials-and-sculptures/table/" target="_blank">Dataset Link</a>
  <br>
</div>
<br>

<div class="usecase-subsection-blurb">
  <i>Dataset 4:</i> Plaques located at the Shrine of Remembrance
  <br>
  <a href="https://data.melbourne.vic.gov.au/explore/dataset/plaques-located-at-the-shrine-of-remembrance/table/" target="_blank">Dataset Link</a>
  <br>
</div>
<br>

# <div class="usecase-section-header"><h4>Project Objective, Overview & Research</h4></div>

In [1]:
#Import core libraries
import requests
import pandas as pd
import numpy as np
import os

import json

import folium
from geopy.geocoders import Nominatim
from ipywidgets import interact, widgets
from IPython.display import display, HTML

In [2]:
def fetch_data(base_url, dataset, api_key, num_records=99, offset=0):
    all_records = []
    max_offset = 9900

    while True:
        if offset > max_offset:
            break

        filters = f'{dataset}/records?limit={num_records}&offset={offset}'
        url = f'{base_url}{filters}&api_key={api_key}'

        try:
            result = requests.get(url, timeout = 10)
            result.raise_for_status()
            records = result.json().get('results')
        except requests.exceptions.RequestException as e:
            raise Exception(f'API request failed: {e}')
        if records is None:
            break
        all_records.extend(records)
        if len(records) < num_records:
            break

        offset += num_records

    df = pd.DataFrame(all_records)
    return df

BASE_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
API_KEY = ''

## Part 1: Pre-Processing

#### Dataset 1: Outdoor artworks

In [3]:
SENSOR_DATASET = 'outdoor-artworks'
outdoor_artworks = fetch_data(BASE_URL, SENSOR_DATASET, API_KEY)
outdoor_artworks.head()

,geo_point_2d,geo_shape,road_segment,location,latitude,asset_id,history,owner_type,description,object_type,art_date,inscription,longitude,company,service_manager,makers,property,classification,title
0,"{'lon': 144.96706152142454, 'lat': -37.8164236...","{'type': 'Feature', 'geometry': {'coordinates'...",Intersection of Swanston Street and Flinders Lane,Intersection of Swanston Street and Flinders L...,144.967062,1087398,None,Owned,"A patinated bronze sculpture, Beyond the Ocean...",Sculpture,1993,None,-37.816424,City of Melbourne,Arts & Culture Branch,"Quinn, Loretta",None,Sculpture,Beyond the Ocean of Existence
1,"{'lon': 144.96773271682704, 'lat': -37.8260713...","{'type': 'Feature', 'geometry': {'coordinates'...",None,Malthouse Plaza approximately 33m NW of CitiPo...,144.967733,1086752,None,Owned,Yellow painted welded plate steel sculpture.<b...,Sculpture,1980,None,-37.826071,City of Melbourne,Arts & Culture Branch,"Robertson Swann, Ron <br>",Malthouse Plaza,Sculpture,Vault
2,"{'lon': 144.97395547173446, 'lat': -37.8124342...","{'type': 'Feature', 'geometry': {'coordinates'...",None,Gordon Reserve approximately 44m West of Depar...,144.973955,1086576,None,Owned,William Stanford's bluestone fountain features...,Fountain,c1870,STANFORD FOUNTAIN / ERECTED 1871 / THIS FOUNTA...,-37.812434,City of Melbourne,Parks Services Branch,"Stanford, William",Gordon Reserve,Sculpture,Stanford Fountain
3,"{'lon': 144.9409520071531, 'lat': -37.82062403...","{'type': 'Feature', 'geometry': {'coordinates'...",None,Buluk Park,144.940952,1638170,<br>,Owned,Shadow Trees is an assemblage construction of ...,Sculpture,2014,None,-37.820624,City of Melbourne,None,"Smart, Sally <br>",Buluk Park,Sculpture,Shadow Trees
4,"{'lon': 144.98110223857506, 'lat': -37.8134099...","{'type': 'Feature', 'geometry': {'coordinates'...",None,Fitzroy Gardens approximately 34m SW of The Pa...,144.981102,1086740,None,Owned,Series of carvings on stump of one of the orig...,Sculpture,1934,"THE FAIRIES' TREE / CARVED BY OLA COHN, MBE AR...",-37.813410,City of Melbourne,Arts & Culture Branch,"COHN, Ola",Fitzroy Gardens,Sculpture,Fairies Tree


In [4]:
outdoor_artworks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   geo_point_2d     202 non-null    object 
 1   geo_shape        202 non-null    object 
 2   road_segment     46 non-null     object 
 3   location         200 non-null    object 
 4   latitude         202 non-null    float64
 5   asset_id         202 non-null    int64  
 6   history          36 non-null     object 
 7   owner_type       200 non-null    object 
 8   description      202 non-null    object 
 9   object_type      202 non-null    object 
 10  art_date         199 non-null    object 
 11  inscription      93 non-null     object 
 12  longitude        202 non-null    float64
 13  company          196 non-null    object 
 14  service_manager  134 non-null    object 
 15  makers           194 non-null    object 
 16  property         151 non-null    object 
 17  classification  

In [5]:
# Replace weird input as None
outdoor_artworks['history'].replace({'<br>':None, '<br> <br><br>':None, '<br><br><br>':None}, inplace=True)

In [6]:
# Splitting into longitude and latitude columns
outdoor_artworks['longitude'] = outdoor_artworks['geo_point_2d'].apply(lambda x: x['lon'])
outdoor_artworks['latitude'] = outdoor_artworks['geo_point_2d'].apply(lambda x: x['lat'])

In [7]:
# Drop unnecessary columns
outdoor_artworks.drop(columns=['geo_point_2d', 'geo_shape'], inplace=True)

#### Dataset 2: Public artworks, fountains and monuments

In [8]:
SENSOR_DATASET = 'public-artworks-fountains-and-monuments'
fountain_monument = fetch_data(BASE_URL, SENSOR_DATASET, API_KEY)
fountain_monument.head()

,asset_type,name,xorg,xsource,address_point,artist,alternate_name,art_date,mel_way_ref,respective_author,structure,co_ordinates,easting,northing
0,Fountain,Coles Fountain,City of Melbourne,MCC - Ortho Image March 2005 - Final,"517 Albert Street, EAST MELBOURNE",Robert Woodward,None,1981,2F_J1,City Of Melbourne,Stainless-steel fountain on bluestone paving,"{'lon': 144.973483976518, 'lat': -37.809770618...",321609.36000,5.813357e+06
1,Monument,Black Swan Memorial Drinking Fountain,City of Melbourne,MCC - Ortho Image March 2005 - Final,"Alexandra Gardens, St Kilda Road, MELBOURNE",Raymond B. Ewers,None,1974,2F_J6,City Of Melbourne,Bluestone drinking fountain with bronze plaque,"{'lon': 144.971824407255, 'lat': -37.819570561...",321486.86700,5.812266e+06
2,Art,Federation Bells,City of Melbourne,MCC - Ground Ortho Image March 2008,"Birrarung Marr, MELBOURNE",Designers Neil McLachlan and Anton Hasell,None,2002,2F_ K6,City of Melbourne,Bronze-alloy bells on galvanised-steel poles,"{'lon': 144.974158800684, 'lat': -37.818697331...",321690.25256,5.812367e+06
3,Art,Federation Bells,City of Melbourne,MCC - Ground Ortho Image March 2008,"Birrarung Marr, MELBOURNE",Designers Neil McLachlan and Anton Hasell,None,2002,2F_ K6,City of Melbourne,Bronze-alloy bells on galvanised-steel poles,"{'lon': 144.974128540287, 'lat': -37.818688233...",321687.56694,5.812368e+06
4,Art,Coat of Arms,City of Melbourne,MCC - Ortho Image March 2005 - Final,"104 Swanston Street, MELBOURNE",City of Melbourne,None,1992,2F_F4,City Of Melbourne,Brass pavement inlay,"{'lon': 144.966444170774, 'lat': -37.815048121...",321002.34200,5.812757e+06


In [9]:
# Drop unnecessary columns
fountain_monument.drop(columns=['mel_way_ref', 'easting', 'northing'], inplace=True)

In [10]:
# Splitting into longitude and latitude columns
fountain_monument['longitude'] = fountain_monument['co_ordinates'].apply(lambda x: x['lon'])
fountain_monument['latitude'] = fountain_monument['co_ordinates'].apply(lambda x: x['lat'])

In [11]:
fountain_monument['respective_author'].unique()

array(['City Of Melbourne', 'City of Melbourne', 'VicUrban',
       'National Gallery of Victoria', None], dtype=object)

In [12]:
# Replace CoM input
fountain_monument['respective_author'].replace({'City Of Melbourne': 'City of Melbourne'}, inplace=True)
fountain_monument['respective_author'].unique()

array(['City of Melbourne', 'VicUrban', 'National Gallery of Victoria',
       None], dtype=object)

#### Dataset 3: Public memorials and sculptures

In [13]:
SENSOR_DATASET = 'public-memorials-and-sculptures'
memorials_sculptures = fetch_data(BASE_URL, SENSOR_DATASET, API_KEY)
memorials_sculptures.head()

,description,title,co_ordinates
0,Arts & Heritage - Sculpture - Memorial Sculpture,South African War Memorial (Memorial to Fallen...,"{'lon': 144.97185679034, 'lat': -37.8259414540..."
1,Arts & Heritage - Memorials - Fountain,Macpherson-Robertson Fountain,"{'lon': 144.972934260924, 'lat': -37.832377458..."
2,Arts & Heritage - Memorials - Drinking Fountain,Councillor William Cook Memorial Drinking Foun...,"{'lon': 144.966190018508, 'lat': -37.785287472..."
3,Arts & Heritage - Sculpture - Bust,Sir Samuel Gillott,"{'lon': 144.972615422561, 'lat': -37.813725736..."
4,Arts & Heritage - Sculpture - Monument,Sir William John Clarke,"{'lon': 144.974344858809, 'lat': -37.813672953..."


In [14]:
# Splitting into longitude and latitude columns
memorials_sculptures['longitude'] = memorials_sculptures['co_ordinates'].apply(lambda x: x['lon'])
memorials_sculptures['latitude'] = memorials_sculptures['co_ordinates'].apply(lambda x: x['lat'])

#### Dataset 4: Plaques located at the Shrine of Remembrance

In [15]:
SENSOR_DATASET = 'plaques-located-at-the-shrine-of-remembrance'
plaques = fetch_data(BASE_URL, SENSOR_DATASET, API_KEY)
plaques.head()

,asset_number,title_of_plaque,description_of_plaque,tree_common_name,tree_scientific_name,date_of_tree_planted,lon,lat,location
0,1498995,2/4TH LIGHT ANTI-AIRCRAFT REGIMENT 9TH DIVISIO...,2/4TH LIGHT ANTI-AIRCRAFT REGIMENT 9TH DIVISIO...,London Plane,Platanus acerifolia,1998-07-09,-37.831930,144.973212,"{'lon': 144.97321170883876, 'lat': -37.8319297..."
1,1499003,THE AUSTRALIAN LEGION OF EX-SERVICEMEN AND WOMEN,1944 1994 FOR ALL WHO SERVE THIS SPOTTED GUM W...,Spotted Gum,Corymbia maculata,1998-07-09,-37.832137,144.972873,"{'lon': 144.9728731147446, 'lat': -37.83213732..."
2,1499042,ROYAL AUSTRALIAN ENGINEERS (TRANSPORTATION SER...,ROYAL AUSTRALIAN ENGINEERS (TRANSPORTATION SER...,River Red Gum,Eucalyptus camaldulensis,1998-07-09,-37.831394,144.972049,"{'lon': 144.97204947974228, 'lat': -37.8313940..."
3,1499163,2/33 BATTALION,DEDICATED TO THOSE WHO SERVED N THE 2/33 BATTA...,UNKNOWN,UNKNOWN UNKNOWN,2000-01-02,-37.828572,144.971739,"{'lon': 144.97173886572406, 'lat': -37.8285718..."
4,1499155,22ND AUSTRALIAN INFANTRY BATTALION,22ND AUSTRALIAN INFANTRY BATTALION 1939-1945,English Elm,Ulmus procera,1998-07-09,-37.828755,144.972360,"{'lon': 144.97236009726893, 'lat': -37.8287553..."


In [16]:
# Splitting into longitude and latitude columns
plaques['longitude'] = plaques['location'].apply(lambda x: x['lon'])
plaques['latitude'] = plaques['location'].apply(lambda x: x['lat'])

In [17]:
# Drop unnecessary columns
plaques.drop(columns=['asset_number', 'lon', 'lat'], inplace=True)

## Part 2: Creating maps

In [38]:
# Use geopy to get the coordinates for Melbourne
geolocator = Nominatim(user_agent="my_map")
location = geolocator.geocode("Melbourne, Australia")
melbourne_coords = (location.latitude, location.longitude)

# Filter data for Melbourne and drop NaN values in 'location' column
outdoor_artworks = outdoor_artworks.dropna(subset=['location'])
outdoor_artworks = outdoor_artworks[(outdoor_artworks['latitude'].notna()) & (outdoor_artworks['longitude'].notna())]

# Create a Folium Map centered around Melbourne
melbourne_map = folium.Map(location=melbourne_coords, zoom_start=12)

# Add markers for all locations
def add_markers(map_obj, data):
    for index, row in data.iterrows():
        folium.Marker(location=[row['latitude'], row['longitude']]).add_to(map_obj)

# Define function to display location and history information
def display_info(filtered_data):
    info_html = ""
    for index, row in filtered_data.iterrows():
        info_html += f"<h3>{row['title']}</h3><p><strong>Location:</strong> {row['location']}</p><p><strong>History:</strong> {row['history']}</p>"
    display(HTML(info_html))

# Define function to update map based on search word
def update_map(search_word):
    global melbourne_map  # Declare melbourne_map as global
    
    # Clear the output display
    display(HTML(""))
    
    # Create a new map object
    melbourne_map = folium.Map(location=melbourne_coords, zoom_start=12)
    
    # If search word is not empty, filter data and display information
    if search_word.strip():
        filtered_data = outdoor_artworks[outdoor_artworks['location'].str.contains(search_word, case=False)]
        # Add filtered markers to the map
        add_markers(melbourne_map, filtered_data)
    else:
        # If search word is blank, add all markers to the map
        add_markers(melbourne_map, outdoor_artworks)
    
    # Display the map
    display(melbourne_map)
    
    # If search word is not empty, display location and history information below the map
    if search_word.strip():
        display_info(filtered_data)

# Create search bar widget
search_bar = widgets.Text(placeholder='Enter search word', description='Search:', disabled=False)

# Display the search bar and map
widgets.interactive(update_map, search_word=search_bar)

interactive(children=(Text(value='', description='Search:', placeholder='Enter search word'), Output()), _dom_…

In [39]:
# Use geopy to get the coordinates for Melbourne
geolocator = Nominatim(user_agent="my_map")
location = geolocator.geocode("Melbourne, Australia")
melbourne_coords = (location.latitude, location.longitude)

# Filter data for Melbourne and drop NaN values in 'location' column
outdoor_artworks = outdoor_artworks.dropna(subset=['location'])
outdoor_artworks = outdoor_artworks[(outdoor_artworks['latitude'].notna()) & (outdoor_artworks['longitude'].notna())]

# Create a unique list of object types for the dropdown widget
object_types = outdoor_artworks['object_type'].unique().tolist()

# Create a Folium Map centered around Melbourne
melbourne_map = folium.Map(location=melbourne_coords, zoom_start=12)

# Add markers for all locations
def add_markers(map_obj, data):
    for index, row in data.iterrows():
        folium.Marker(location=[row['latitude'], row['longitude']]).add_to(map_obj)

# Define function to display location and history information
def display_info(filtered_data):
    info_html = ""
    for index, row in filtered_data.iterrows():
        info_html += f"<h3>{row['title']}</h3><p><strong>Location:</strong> {row['location']}</p><p><strong>History:</strong> {row['history']}</p>"
    display(HTML(info_html))

# Define function to update map based on search word and object type
def update_map(search_word, object_type):
    global melbourne_map  # Declare melbourne_map as global
    
    # Clear the output display
    display(HTML(""))
    
    # Create a new map object
    melbourne_map = folium.Map(location=melbourne_coords, zoom_start=12)
    
    # If search word is not empty, filter data based on search word
    if search_word.strip():
        filtered_data = outdoor_artworks[outdoor_artworks['location'].str.contains(search_word, case=False)]
    else:
        filtered_data = outdoor_artworks.copy()  # Use all data if search word is empty
    
    # If object type is selected, filter data based on object type
    if object_type != "All":
        filtered_data = filtered_data[filtered_data['object_type'] == object_type]
    
    # Add filtered markers to the map
    add_markers(melbourne_map, filtered_data)
    
    # Display the map
    display(melbourne_map)
    
    # If search word is not empty, display location and history information below the map
    if search_word.strip():
        display_info(filtered_data)

# Create search bar widget
search_bar = widgets.Text(placeholder='Enter search word', description='Search:', disabled=False)

# Create dropdown widget for object type
object_type_dropdown = widgets.Dropdown(options=["All"] + object_types, description='Object Type:', disabled=False)

# Display the search bar, dropdown list, and map
widgets.interactive(update_map, search_word=search_bar, object_type=object_type_dropdown)

interactive(children=(Text(value='', description='Search:', placeholder='Enter search word'), Dropdown(descrip…